# Full Sherpa test for marginalization setup

## Fitting rl, flux, epoch and hstp1-4 being free

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.constants import G

os.chdir('../')
from config import CONFIG_INI
from limb_darkening import limb_dark_fit
import margmodule as marg

from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.fit import Fit
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2
from sherpa.plot import FitPlot

### Set up data paths

In [ ]:
localDir = CONFIG_INI.get('data_paths', 'local_path')
outDir = os.path.join(localDir, CONFIG_INI.get('data_paths', 'output_path'))
curr_model = CONFIG_INI.get('data_paths', 'current_model')
dataDir = os.path.join(localDir, os.path.join(localDir, CONFIG_INI.get('data_paths', 'data_path')), curr_model)

### Read in data

In [ ]:
# Read in the txt file for the lightcurve data
x, y, err, sh = np.loadtxt(os.path.join(dataDir, 'W17_white_lightcurve_test_data.txt'), skiprows=7, unpack=True)
wavelength = np.loadtxt(os.path.join(dataDir, 'W17_wavelength_test_data.txt'), skiprows=3)

tzero = x[0]
flux0 = y[0]

print("x.shape: {}".format(x.shape))
print("y.shape: {}".format(y.shape))
print("err.shape: {}".format(err.shape))
print("sh.shape: {}".format(sh.shape))
print("wvln.shape: {}".format(wavelength.shape))

### Read and set up planet parameters

Not importing `rl`, `epoch`, `inclin`, `ecc`, `omega` and `Per` sinc they will be read by the model directly from the configfile.

Actually, I do need to import `Per` because I need it for the calculation of MsMpR.

In [ ]:
Per = CONFIG_INI.getfloat('planet_parameters', 'Per') * u.d
Per = Per.to(u.s)
constant1 = ((G * np.square(Per)) / (4 * np.square(np.pi))) ** (1 / 3)
aor = CONFIG_INI.getfloat('planet_parameters', 'aor')    # this is unitless -> "distance of the planet from the star (meters)/stellar radius (meters)"
MsMpR = (aor / constant1) ** 3.
print("MsMpR: {}".format(MsMpR))

### Systematic model parameters

The starting parameters for the systematic models are all 0 by default, hence I am not setting them here explicitly.

### Limb darkening

In [ ]:
# Limb darkening
M_H = CONFIG_INI.getfloat('limb_darkening', 'metallicity')    # metallicity
Teff = CONFIG_INI.getfloat('limb_darkening', 'Teff')   # effective temperature
logg = CONFIG_INI.getfloat('limb_darkening', 'logg')   # log(g), gravitation

# Define limb darkening directory, which is inside this package
limbDir = os.path.join('..', 'Limb-darkening')
ld_model = CONFIG_INI.get('limb_darkening', 'ld_model')
grat = CONFIG_INI.get('technical_parameters', 'grating')
_uLD, c1, c2, c3, c4, _cp1, _cp2, _cp3, _cp4, _aLD, _bLD = limb_dark_fit(grat, wavelength, M_H,
                                                                         Teff, logg, limbDir, ld_model)
print("\nThe four cs: {}, {}, {}, {}".format(c1, c2, c3, c4))

### Select systematic grid

For testing purposes, I will only pick one of them and I will set it by hand so that we know what is going on.

In [ ]:
# p0 =          [0,    1,     2,      3,     4,    5,    6,    7,  8,  9,  10, 11, 12,  13,    14,    15,    16,    17,     18,      19,      20,      21   ]
# p0 = np.array([rl, flux0, epoch, inclin, MsMpR, ecc, omega, Per, T0, c1, c2, c3, c4, m_fac, HSTP1, HSTP2, HSTP3, HSTP4, xshift1, xshift2, xshift3, xshift4])
nparams = 22
# 1 in the grid means the parameter is fixed, 0 means it is free.
systematics = np.ones((nparams))
# Choice of only rl and flux free, first test
systematics[0] = 0
systematics[1] = 0
systematics[2] = 0
systematics[14:18] = 0

print("Systematics: {}".format(systematics))

### Set up Sherpa data object

In [ ]:
# Instantiate a data object
data = Data1D('Data', x, y, staterror=err)
print(data)

In [ ]:
# Plot the data with Sherpa
dplot = DataPlot()
dplot.prepare(data)
dplot.plot() 

### Set up Sherpa model object

In [ ]:
# Define the model
tmodel = marg.Transit(tzero, MsMpR, c1, c2, c3, c4, flux0, name="testmodel", sh=sh)
print(tmodel)

### Freeze some parameters

Most parameters are currently thawed by default. `omga`, `period`, `tzero` and all the limb darkening parameters `c1`-`c4` have `alwaysfrozen=True` in the class, so that we don't have to explicitly set them to frozen every time. And we'll still need the systematics grid to loop through the different systematic models, but I am leaving that out for now.

In [ ]:
# Freeze all but rl and flux
tmodel.epoch.frozen = False
tmodel.inclin.frozen = True
tmodel.msmpr.frozen = True
tmodel.ecc.frozen = True
tmodel.m_fac.frozen = True
tmodel.hstp1.frozen = False
tmodel.hstp2.frozen = False
tmodel.hstp3.frozen = False
tmodel.hstp4.frozen = False
tmodel.xshift1.frozen = True
tmodel.xshift2.frozen = True
tmodel.xshift3.frozen = True
tmodel.xshift4.frozen = True

print(tmodel)

In [ ]:
# Plot the model
mplot = ModelPlot()
mplot.prepare(data, tmodel)
mplot.plot()

Turns out there are different ways of evaulating a model on an arbitrary grid, check out:  
https://sherpa.readthedocs.io/en/4.11.0/evaluation/examples.html#examples

### Set up statistics and optimizer

In [ ]:
stat = Chi2()
opt = LevMar()

### Set up fit model and make the fit

In [ ]:
tfit = Fit(data, tmodel, stat=stat, method=opt)    # Instantiate fit object
tres = tfit.fit()     # do the fit
print(tres)

if not tres.succeeded: print(tres.message)

In the row `parvals`, we can see that a high number precision is given when dealing with the data. This means that when we see less digits in some of the displays, that will only be becuase of pretty printing.

We can also display the results in a formatted way:

In [ ]:
print(tres.format())

### Plot fit over data

In [ ]:
# Plot fit over data
fplot = FitPlot()
mplot.prepare(data, tmodel)
fplot.prepare(dplot, mplot)
fplot.plot()

### Error analysis

The default error estimation analysis method is `covariance`.

In [ ]:
print(tfit.estmethod)

In [ ]:
# Run the error analysis
errors = tfit.est_errors()

In [ ]:
# Display the errors
print(errors.format())

This calculates the one-sigma (68.3%) limits for each thawed parameter and the error range can be changed with the `sigma` parameter (https://sherpa.readthedocs.io/en/4.11.0/fit/index.html#changing-the-error-bounds).

We can directly access the errors:

In [ ]:
# Just showing the full (unformatted) output
print(errors)

In [ ]:
# Access the errors in dictionary
dvals = zip(errors.parnames, errors.parvals, errors.parmins,
errors.parmaxes)
pvals = {d[0]: {'val': d[1], 'min': d[2], 'max': d[3]} for d in dvals}

print(pvals['testmodel.rl'])

In order to save time, we an restrict the error calculation to only the paremeters we need:

In [ ]:
rl_err = tfit.est_errors(parlist=(tmodel.rl,))
print(rl_err)

More on errors, also on how to investigate error surfaces or do contour plots is described here:  
https://sherpa.readthedocs.io/en/4.11.0/quick.html#error-analysis  
and here:  
https://sherpa.readthedocs.io/en/4.11.0/fit/index.html#estimating-errors

## Results for thawed rl, flux, epoch and hstp 1-4

Lets print the value and error results for the fitting of `rl` and `flux`, so that we can compare them with the IDL results.

In [ ]:
ans = dict(zip(tres.parnames, tres.parvals))

print('rl after fit:')
print(pvals['testmodel.rl'])

print('\nflux after fit:')
print(pvals['testmodel.flux'])

print('\nepoch after fit:')
print(pvals['testmodel.epoch'])

print('\nhstp1 after fit:')
print(pvals['testmodel.hstp1'])

print('\nhstp2 after fit:')
print(pvals['testmodel.hstp2'])

print('\nhstp3 after fit:')
print(pvals['testmodel.hstp3'])

print('\nhstp4 after fit:')
print(pvals['testmodel.hstp4'])

### Resetting the model parameters

In [ ]:
print(tmodel)

We can reset the model parameters to their initial values.

In [ ]:
tmodel.reset()
print(tmodel)